### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime

# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-14 21:18:08


In [3]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"Obsidian path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
Obsidian path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [7]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [8]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [9]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [10]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [11]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  09:18 PM


In [12]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.27,1.26,1.28,1.23,1.52,0.0,2025-02-14
ADVANC,299.00,295.00,304.00,193.00,307.00,-1.64,2025-02-14
AEONTS,108.00,107.00,108.50,103.50,168.50,0.0,2025-02-14
AH,14.00,14.00,14.10,13.70,26.25,0.72,2025-02-14
AIE,0.74,0.74,0.75,0.73,1.51,-1.33,2025-02-14
AIMIRT,10.60,10.50,10.60,10.20,11.20,0.95,2025-02-14
AIT,4.60,4.54,4.68,3.74,5.05,-1.29,2025-02-14
AJ,2.38,2.32,2.44,2.26,7.15,2.59,2025-02-14
AMATA,24.50,24.40,24.90,20.10,31.00,-0.41,2025-02-14
ANAN,0.42,0.41,0.43,0.40,0.94,2.44,2025-02-14
AOT,47.00,46.50,54.25,53.25,67.50,-13.76,2025-02-14
AP,7.80,7.75,7.90,7.50,11.20,-0.64,2025-02-14
ASIAN,6.95,6.85,7.00,6.35,11.20,1.46,2025-02-14
ASK,7.60,7.55,7.75,7.40,19.40,0.0,2025-02-14
ASP,2.30,2.28,2.30,2.12,2.88,0.88,2025-02-14
ASW,7.50,7.50,7.65,7.35,8.45,-2.6,2025-02-14
AWC,3.04,2.98,3.06,2.88,4.58,1.33,2025-02-14
BA,19.50,19.40,19.80,15.10,26.00,1.56,2025-02-14
BAM,6.00,5.80,6.00,5.35,10.40,0.84,2025-02-14
BANPU,4.52,4.52,4.64,4.38,7.50,-0.88,2025-02-14
BAY,22.70,22.50,23.00,22.20,29.00,

In [13]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  09:23 PM


In [14]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
25,BCT,-,0.00,0.00,53.00,76.75,0.0,2025-02-14


In [15]:
out_df.shape

(221, 8)

In [16]:
out_df = out_df[out_df['name'] != 'BCT']

In [17]:
out_df = out_df[out_df['name'] != 'LPF']

In [18]:
out_df = out_df[out_df['name'] != 'TYCN']

In [19]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [20]:
out_df = out_df[out_df['name'] != 'MST']

In [21]:
out_df.shape

(217, 8)

In [22]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [23]:
sorted_out_df = out_df.sort_values('percent')

In [24]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
10,AOT,47.00,46.50,54.25,53.25,67.50,-13.76,2025-02-14
49,CPN,48.75,48.75,52.00,49.25,68.75,-6.70,2025-02-14
70,GFPT,8.30,8.20,8.65,8.50,13.40,-3.49,2025-02-14
160,SINGER,5.80,5.75,6.10,5.95,12.70,-3.33,2025-02-14
125,ORI,2.90,2.88,3.02,2.94,8.45,-2.68,2025-02-14
89,IP,2.16,2.14,2.20,2.16,7.30,-1.82,2025-02-14
143,RBF,4.74,4.68,4.86,4.76,12.70,-1.66,2025-02-14
97,KEX,1.25,1.25,1.30,1.27,6.50,-1.57,2025-02-14
147,RS,0.79,0.79,0.84,0.80,7.35,-1.25,2025-02-14
198,TPIPL,0.93,0.92,0.95,0.94,1.45,-1.06,2025-02-14


In [25]:
sorted_out_df.query('name == "MCS"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
116,MCS,6.45,6.4,6.6,5.8,8.2,-0.77,2025-02-14


In [26]:
out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
10,AOT,47.00,46.50,54.25,53.25,67.50,-13.76,2025-02-14
49,CPN,48.75,48.75,52.00,49.25,68.75,-6.70,2025-02-14
70,GFPT,8.30,8.20,8.65,8.50,13.40,-3.49,2025-02-14
72,GLOBAL,8.10,7.80,8.10,7.85,17.40,1.89,2025-02-14
89,IP,2.16,2.14,2.20,2.16,7.30,-1.82,2025-02-14
97,KEX,1.25,1.25,1.30,1.27,6.50,-1.57,2025-02-14
108,LHK,3.36,3.32,3.38,3.34,4.26,0.00,2025-02-14
125,ORI,2.90,2.88,3.02,2.94,8.45,-2.68,2025-02-14
143,RBF,4.74,4.68,4.86,4.76,12.70,-1.66,2025-02-14
147,RS,0.79,0.79,0.84,0.80,7.35,-1.25,2025-02-14


In [27]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [28]:
out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [29]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [30]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [31]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [32]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-14 21:23:28
